In [2]:
import sys
sys.path.append('./maxine/')
import tensorflow as tf
print(tf.__version__)

# Helper libraries
from tensorflow import keras
from keras import mixed_precision
import numpy as np
import time
import maxine
import sys
from maxine.layers import SoftMaxMin_Pruned_v02
import keras.backend as k
from maxine.tools import pruning_tool_MLP_v3
from maxine.tools import pruning_tool_MLP_v4_1
from maxine.tools import pruning_tool_MLP_v4_2
import tensorflow_model_optimization as tfmot
from keras import regularizers
from keras.callbacks import EarlyStopping

2.13.0


In [3]:
# LOAD DATA
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

# convert to float and normalize
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

# Validation data
x_val = x_train[-5000:]
y_val = y_train[-5000:]

x_train = x_train[:-5000]
y_train = y_train[:-5000]

In [4]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

In [6]:
baselineModel = tf.keras.models.load_model('./models/Lenet300100_Fashion')
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.975, nesterov=True)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_2.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=40, p=0, pList=[3, 0.1], optimizer_config=tf.optimizers.serialize(optimizer), extreme=True, preLoad=False, activLists=[], activCounts=[])

Training morphological model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min_1 (SoftMaxMin  (55000, 300)              235200    
 )                                                               
                                                                 
 dense_1 (Dense)             (55000, 10)               3000      
                                                                 
Total params: 238200 (930.47 KB)
Trainable params: 238200 (930.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1719/1719 - 10s - loss: 0.8977 - accuracy: 0.7188 - val_loss: 0.5168 - val_accuracy: 0.8208 - 10s/epoch - 6ms/step
Epoch 2/30
1719/1719 - 8s - loss: 0.4775 - accuracy: 0.8341 - val_loss: 0.4467 - val_accuracy: 0.8390 - 8s/epoch - 4ms/step
Epoch 3/30
1719/1719 - 8s - loss: 0.4277 - accuracy: 0.8502 - 

In [7]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.975, nesterov=True)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer='adam', 
            metrics=['accuracy'],
            jit_compile=True)
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)

Epoch 1/40
430/430 [==============================] - 4s 7ms/step - loss: 0.2513 - accuracy: 0.9092 - val_loss: 0.3319 - val_accuracy: 0.8814
Epoch 2/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2490 - accuracy: 0.9106 - val_loss: 0.3339 - val_accuracy: 0.8800
Epoch 3/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2485 - accuracy: 0.9099 - val_loss: 0.3325 - val_accuracy: 0.8814
Epoch 4/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2476 - accuracy: 0.9106 - val_loss: 0.3316 - val_accuracy: 0.8808
Epoch 5/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2468 - accuracy: 0.9107 - val_loss: 0.3346 - val_accuracy: 0.8818
Epoch 6/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2462 - accuracy: 0.9107 - val_loss: 0.3398 - val_accuracy: 0.8778
Epoch 7/40
430/430 [==============================] - 2s 4ms/step - loss: 0.2456 - accuracy: 0.9106 - val_loss: 0.3345 - val_accuracy: 0.8814
Epoch 

[0.3571011424064636, 0.8794000148773193]